In [3]:
import numpy as np
import math

# Define the gamma_g function (the model formula)
def calculate_gamma_g(API, T, P, alpha):
    """
    Calculate gamma_g using the provided formula.
    
    Parameters:
    API (float): API gravity of the oil
    T (float): Temperature in Rankine (°R)
    P (float): Pressure (psia or atm)
    alpha (list): List of the parameters [alpha_1, alpha_2, alpha_3, alpha_4, alpha_5, alpha_6, alpha_7]
    
    Returns:
    float: The calculated gas density gamma_g
    """
    alpha_1, alpha_2, alpha_3, alpha_4, alpha_5, alpha_6, alpha_7 = alpha
    
    # Compute the terms
    term1 = (API ** alpha_1) / ((T - 460) ** alpha_2)  # (API^alpha_1) / (T-460)^alpha_2
    term2 = alpha_4 - (alpha_5 - alpha_6 * math.log10(P)) ** alpha_7  # alpha_4 - (alpha_5 - alpha_6*log(P))^alpha_7
    
    # Combine the terms and raise to the power of alpha_3
    gamma_g = (term1 * 10 ** term2) ** alpha_3
    
    return gamma_g

# Define the objective function (Sum of Squared Errors or SSE)
def objective_function(alpha, API_data, T_data, P_data, GOR_data):
    """
    Objective function to minimize: the sum of squared errors between predicted and actual GOR values.
    
    Parameters:
    alpha (list): The parameters [alpha_1, alpha_2, alpha_3, alpha_4, alpha_5, alpha_6, alpha_7]
    API_data (array): Array of API gravity values
    T_data (array): Array of temperature values in Rankine
    P_data (array): Array of pressure values in psia
    GOR_data (array): Array of observed gas-to-oil ratio values
    
    Returns:
    float: The sum of squared errors (SSE)
    """
    predicted_GOR = np.array([calculate_gamma_g(API, T, P, alpha) for API, T, P in zip(API_data, T_data, P_data)])
    error = np.sum((predicted_GOR - GOR_data) ** 2)  # Sum of squared errors
    return error

# Ant Colony Optimization (ACO) Algorithm
def aco_optimize(API_data, T_data, P_data, GOR_data, n_ants=20, n_iterations=100, alpha_params=None, pheromone_decay=0.95, pheromone_influence=1.0):
    """
    Perform Ant Colony Optimization (ACO) to optimize the gamma_g formula parameters.
    
    Parameters:
    API_data (array): Array of API gravity values
    T_data (array): Array of temperature values in Rankine
    P_data (array): Array of pressure values in psia
    GOR_data (array): Array of observed gas-to-oil ratio values
    n_ants (int): Number of ants (solutions)
    n_iterations (int): Number of iterations (generations)
    alpha_params (list): Initial values of the parameters [alpha_1, alpha_2, alpha_3, alpha_4, alpha_5, alpha_6, alpha_7]
    pheromone_decay (float): Decay factor for pheromone
    pheromone_influence (float): Influence factor for pheromone in solution generation
    
    Returns:
    tuple: Best parameter values (alpha) and best error
    """
    # Initial pheromone levels (for each parameter)
    pheromone = np.ones((n_ants, 7))  # 7 parameters: alpha_1 to alpha_7
    
    # Parameters
    alpha = np.array(alpha_params) if alpha_params is not None else np.random.rand(7)  # Initial alpha params
    
    # Best solution initialization
    best_alpha = alpha.copy()
    best_error = float('inf')
    
    # Iteration loop
    for iteration in range(n_iterations):
        solutions = []
        errors = []
        
        # Each ant explores a potential solution
        for ant in range(n_ants):
            # Explore new solution based on pheromone levels and random exploration
            new_alpha = alpha * (1 + pheromone_influence * (np.random.rand(7) - 0.5))  # Perturbation of alpha using pheromone influence
            # Evaluate the solution
            error = objective_function(new_alpha, API_data, T_data, P_data, GOR_data)
            solutions.append(new_alpha)
            errors.append(error)
            
            # Update best solution if a better one is found
            if error < best_error:
                best_error = error
                best_alpha = new_alpha
        
        # Update pheromones based on the solutions and errors
        pheromone *= pheromone_decay  # Decay the pheromone
        for ant in range(n_ants):
            pheromone[ant] += pheromone_influence / (errors[ant] + 1e-6)  # Inverse of error (larger error -> less pheromone)
        
        # Print the best error and alpha values found in this iteration
        print(f"Iteration {iteration + 1}, Best Error: {best_error}, Best Alpha: {best_alpha}")
    
    return best_alpha, best_error

# Example usage:
# Example data: API, Temperature, Pressure, and observed GOR
API_data = np.array([35, 40, 45, 50, 55])
T_data = np.array([520, 530, 540, 550, 560])
P_data = np.array([3000, 3500, 4000, 4500, 5000])
GOR_data = np.array([400, 420, 430, 460, 480])

# Initial alpha parameters (just for initialization, values will be optimized)
initial_alpha_params = [0.989, 0.172, 1.2255, 2.8869, 14.1811, 3.3053, 0.5]

# Run ACO optimization
best_alpha, best_error = aco_optimize(API_data, T_data, P_data, GOR_data, n_ants=20, n_iterations=100, alpha_params=initial_alpha_params)

print(f"Optimized alpha values: {best_alpha}")
print(f"Best error: {best_error}")


Iteration 1, Best Error: 60793.40859992501, Best Alpha: [ 1.21122887  0.21933498  1.40927664  2.01000623 15.62869144  2.64295034
  0.29105497]
Iteration 2, Best Error: 60793.40859992501, Best Alpha: [ 1.21122887  0.21933498  1.40927664  2.01000623 15.62869144  2.64295034
  0.29105497]
Iteration 3, Best Error: 60793.40859992501, Best Alpha: [ 1.21122887  0.21933498  1.40927664  2.01000623 15.62869144  2.64295034
  0.29105497]
Iteration 4, Best Error: 60793.40859992501, Best Alpha: [ 1.21122887  0.21933498  1.40927664  2.01000623 15.62869144  2.64295034
  0.29105497]
Iteration 5, Best Error: 60793.40859992501, Best Alpha: [ 1.21122887  0.21933498  1.40927664  2.01000623 15.62869144  2.64295034
  0.29105497]
Iteration 6, Best Error: 60793.40859992501, Best Alpha: [ 1.21122887  0.21933498  1.40927664  2.01000623 15.62869144  2.64295034
  0.29105497]
Iteration 7, Best Error: 60793.40859992501, Best Alpha: [ 1.21122887  0.21933498  1.40927664  2.01000623 15.62869144  2.64295034
  0.29105497]

C:\Users\CALEB\AppData\Local\Temp\ipykernel_29928\3709352188.py:22: RuntimeWarning: invalid value encountered in scalar power
  term2 = alpha_4 - (alpha_5 - alpha_6 * math.log10(P)) ** alpha_7  # alpha_4 - (alpha_5 - alpha_6*log(P))^alpha_7
